**<font color = black size=6>实验七:贝叶斯分布</font>**

In [15]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

**<font color = blue size=4>第一部分:实验任务</font>**

1.朴素贝叶斯

<span style="color:purple">该数据集(train_mushroom.csv)为分类数据集，为蘑菇的特征信息以及是否有毒。包括了13个属性值以及一个类别属性(即为label类型,代表是否有毒)</span>

<span style="color:purple">1) 将训练数据集'train_mushroom.csv'和'test_mushroom.csv'载入并转换为你需要的格式</span>

In [16]:
train_frame = pd.read_csv('train_mushroom.csv')
test_frame = pd.read_csv('test_mushroom.csv')
train = np.array(train_frame)
test = np.array(test_frame)
train_frame['cap-shape'][0]

's'

<span style="color:purple">2) 计算每个标记y对应的先验概率P(y)
$$P(y)=\frac{|D_y|}{|D|}$$
    其中$D_y$为标记为y的样本集合；$|D_y|$为标记为y的集合的样本个数；D为所有样本集合；|D|为所有样本个数

</span>

In [17]:
#计算先验概率
counter = Counter(train[:,train.shape[1]-1])
a=np.unique(train[:,train.shape[1]-1])
py1=counter['p']/len(train)
py2=counter['e']/len(train)


<span style="color:purple">3) 计算数据集中的每个属性的非重复属性值$x_i$对应标记y的条件概率$P(x_i│y)$,
    $$P(x_i│y)=\frac{|D_{x_i,y}|}{|D_y|}$$
$D_{x_i,y}$为标记为y，属性为$x_i$的样本集合；$|D_{x_i,y}|$为该样本个数
</span>

In [18]:
#首先操作数据集D，遍历D中的每个属性，将数据集中每个属性的非重复值取出
p1=[]
p2=[]
for i in range(train.shape[1]-1):
    counter1 = Counter(train[:,i])
    a1=np.unique(train[:,i])
    ptem1=[]
    ptem2=[]
    for j in range(len(a1)):
        
        ptem11=0
        ptem22=0
        for k in range(len(train)):
            if train[k,i]==a1[j]:
                if train[k,train.shape[1]-1]=='p':
#                     print("hello")
                    ptem11+=1
                else:
                    ptem22+=1
            else:
                continue
#         print(ptem11)
#         print(ptem22)
        ptem1.append(ptem11/counter['p'])
        ptem2.append(ptem22/counter['e'])
#     print(np.array(ptem1).reshape(1,len(ptem1)).shape)
    df1 = pd.DataFrame(np.array(ptem1).reshape(1,len(ptem1)), columns=list(a1))
    df2 = pd.DataFrame(np.array(ptem2).reshape(1,len(ptem2)), columns=list(a1))
    p1.append(df1)
    p2.append(df2)
# print(float(p2[1]['y']))

#将所有标记相同的数据集取出分为若干数据集

#现以标记为p的数据集Dp为例子，依次对Dp的每个属性值进行遍历，以属性cap-shape为例子介绍，Dp中cap-shape的非重复值集合
#为['b' 'c' 'f' 'k' 's' 'x' 'y']，
#计算数据集中D0中属性值分别为0和1的条件概率P（cap-shape='b'|label='p'）,...,P（cap-shape='y'|label='p'），
#上述对cap-shape属性操作完成后，按照同样的步骤对Dp中的剩余12个属性进行同样的操作



#得到数据集Dp中条件概率集合后，以同样的操作对剩余数据集进行操作


<span style="color:purple">4) 编写函数，给定样本a后以及标记y后计算其后验概率    
    输入：样本a，标记y  
    输出：样本a对应标记y的后验概率  
    计算后验概率公式:
$P(y)\prod_{i=1}^{d}P(x_i|y)$   
    
<span style="color:purple">例:  
    输入:[k,y,n,f,s,c,n,b,o,e,w,v,d]	p  
    输出:P(label='p')$*$P(cap-shape='k'|label='p')$*...*$P(habitat='d'|label='p')</span>

In [19]:
def pro(a,index):
    total=1
#     print(len(p1))
#     print(len(a))
    if index=='p':
        for i in range(len(a)):
            
            if(a[i] in p1[i]):
                total*=float(p1[i][a[i]])
            else:
                total=0
                break
        total*=py1
    if index=='e':
        for i in range(len(a)):
#             print(a[i])
            if(a[i] in p1[i]):
                total*=float(p2[i][a[i]])
            else:
                total=0
                break
        total*=py2
    return total

<span style="color:purple">5) 对测试集中的每个样本a，利用上个步骤所编写的函数分别所有可能的标记对应的后验概率，后验概率更大的对应标记即为预测标记，最后与测试集本身标记进行比较计算出准确率</span>

In [20]:
# len(train[0,:])
yuce=[]
number=0
for i in range(len(test)):
    if pro(test[i,0:test.shape[1]-1],'p')>pro(test[i,0:test.shape[1]-1],'e'):
        yuce.append('p')
        if test[i,test.shape[1]-1]=='p':
            number+=1
    else:
        yuce.append('e')
        if test[i,test.shape[1]-1]=='e':
            number+=1
# print(yuce)
# print(train[:,13]==yuce)
number/len(test)


0.34

2.引入拉普拉斯平滑

<span style="color:purple">1) 请首先判断是否有某个属性值和某个类没有在训练集中同时出现而使得条件概率为零。如果无，则无需进行下列实验；如果有，请在上个实验的基础上引入拉普拉斯平滑</span>

In [9]:
# print(p2[0])
# for i in p2[0]:
#     print(float(p2[0][i])==0)
for i in range(len(p1)):
    for j in p1[i]:
        if(float(p1[i][j])==0):
            print("有")
for i in range(len(p2)):
    for j in p2[i]:
        if(float(p2[i][j])==0):
            print("有")        

有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有
有


<span style="color:purple">2) 引入拉普拉斯平滑后计算每个标记y对应的先验概率P(y)
$$P(y)=\frac{|D_y|+1}{|D|+N}$$
    其中$D_y$为标记为y的样本集合；$|D_y|$为标记为y的集合的样本个数；D为所有样本集合；|D|为所有样本个数;N为标记取值的个数

</span>

In [10]:
#计算先验概率
py1=(counter['p']+1)/(len(train)+2)
py2=(counter['e']+1)/(len(train)+2)

<span style="color:purple">3) 计算数据集中的每个属性的非重复属性值$x_i$对应标记y的条件概率$P(x_i│y)$,
    $$P(x_i│y)=\frac{|D_{x_i,y}|+1}{|D_y|+N_i}$$
$D_{x_i,y}$为标记为y，属性为$x_i$的样本集合；$|D_{x_i,y}|$为该样本个数;$N_i$为第i个属性取值的个数
</span>

In [11]:
#首先操作数据集D，遍历D中的每个属性，将数据集中每个属性的非重复值取出
p1=[]
p2=[]
for i in range(train.shape[1]-1):
    counter1 = Counter(train[:,i])
    a1=np.unique(test[:,i])
    ptem1=[]
    ptem2=[]
    for j in range(len(a1)):
        
        ptem11=1
        ptem22=1
        for k in range(len(train)):
            if train[k,i]==a1[j]:
                if train[k,train.shape[1]-1]=='p':
#                     print("hello")
                    ptem11+=1
                else:
                    ptem22+=1
            else:
                continue
#         print(ptem11)
#         print(ptem22)
        ptem1.append(ptem11/(counter['p']+len(a1)))
        ptem2.append(ptem22/(counter['e']+len(a1)))
#     print(np.array(ptem1).reshape(1,len(ptem1)).shape)
    df1 = pd.DataFrame(np.array(ptem1).reshape(1,len(ptem1)), columns=list(a1))
    df2 = pd.DataFrame(np.array(ptem2).reshape(1,len(ptem2)), columns=list(a1))
    p1.append(df1)
    p2.append(df2)


#将所有标记相同的数据集取出分为若干数据集

#现以标记为p的数据集Dp为例子，依次对Dp的每个属性值进行遍历，以属性cap-shape为例子介绍，Dp中cap-shape的非重复值集合
#为['b' 'c' 'f' 'k' 's' 'x' 'y']，计算数据集中D0中属性值分别为0和1的条件概率P（cap-shape='b'|label='p'）,...,P（cap-shape='y'|label='p'），
#计算条件概率时需要注意引入拉普拉斯平滑
#上述对cap-shape属性操作完成后，按照同样的步骤对Dp中的剩余12个属性进行同样的操作



#得到数据集Dp中条件概率集合后，以同样的操作对剩余数据集进行操作


<span style="color:purple">4) 对测试集中的每个样本a，将先验概率和条件概率替换为使用拉普拉斯平滑后的先验概率和条件概率，重新使用上个步骤所编写的函数分别计算所有可能的标记对应的后验概率，后验概率更大的对应标记即为预测标记，最后与测试集本身标记进行比较计算出准确率</span>

In [12]:
def new_pro(a,index):
    total=1
#     print(len(p1))
#     print(len(a))
    if index=='p':
        for i in range(len(a)):
            
#             if(a[i] in p1[i]):
            total*=float(p1[i][a[i]])
#             else:
#                 total=0
#                 break
        total*=py1
    if index=='e':
        for i in range(len(a)):
#             print(a[i])
#             if(a[i] in p1[i]):
            total*=float(p2[i][a[i]])
#             else:
#                 total=0
#                 break
        total*=py2
    return total

In [13]:
yuce=[]
number=0
for i in range(len(test)):
    if new_pro(test[i,0:test.shape[1]-1],'p')>new_pro(test[i,0:test.shape[1]-1],'e'):
        yuce.append('p')
        if test[i,test.shape[1]-1]=='p':
            number+=1
    else:
        yuce.append('e')
        if test[i,test.shape[1]-1]=='e':
            number+=1
# print(yuce)
# print(train[:,13]==yuce)
number/len(test)

0.58

**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告下周五实验课(5月6号前)上课前提交报告  
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验七(贝叶斯分布)的实验报告上交地址:https://workspace.jianguoyun.com/inbox/collect/87e296bcb96c40b8a6aaad14c628feec/submit

三、课堂课件获取地址:https://www.jianguoyun.com/p/DQlpUFYQp5WhChiS_q0E  
实验内容获取地址:https://www.jianguoyun.com/p/DbKbP-AQp5WhChi1sa0E